## part4_invest_metrics

除了无常损失之外，还可以引入一些其他的金融metrics来衡量头寸的收益。

### ROI

$$ ROI = \frac {Net Profit} {Net Cost} = \frac { amount_{1} - amount_{0} } { amount_{0} }$$


处理成年华收益率：

$$ r_{annualized} = (1 + roi) ^ { \frac {365 } {days}} - 1 $$

### 




In [147]:
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
reload(swapData)

query = swapData.SwapDataQuery()

In [148]:
position_id = 466521
pool_id = "0x4e0924d3a751be199c426d52fb1f2337fa96f736"

data = query.query_position(position_id=position_id)

data

,id,owner,liquidity,depositedToken0,depositedToken1,withdrawnToken0,withdrawnToken1,collectedFeesToken0,collectedFeesToken1,feeGrowthInside0LastX128,feeGrowthInside1LastX128,token0.id,token1.id,tickLower.id,tickUpper.id,transaction.id
0,466521,0xc3830172ef4e76e2a35f2b540579fd4507a77cb7,8027697143087340808,0,12106.759809,0,0,0,0,3657598038082877278892900360302839002720,4443699903658259045286344006,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x4e0924d3a751be199c426d52fb1f2337fa96f736#-27...,0x4e0924d3a751be199c426d52fb1f2337fa96f736#-27...,0x46f8f5f403dbf1608a8bdf725957ee4ffbb53a340d06...


In [149]:
data["depositedToken0"][0]

'0'

In [150]:
# 这个池子是LUSD/USDC的池子, token0: lusd; token1: usdc
from datetime import datetime
begin = datetime(2023, 3, 11)
end = datetime(2023, 4, 15)
position_id = 466521
pool_id = "0x4e0924d3a751be199c426d52fb1f2337fa96f736"

decimal_usdc = 6
decimal_lusd = 18

# 计算upper price和lower price
position_data = query.query_position(position_id=position_id)
position_data
tick_i_l = int(position_data["tickLower.id"][0][43:])
tick_i_u = int(position_data["tickUpper.id"][0][43:])
price_l = (1.0001 ** tick_i_l)
price_u = (1.0001 ** tick_i_u)
# p_real是根据decimal转换之后的值
price_l_real = (1.0001 ** tick_i_l)*(10 ** 12)
price_u_real = (1.0001 ** tick_i_u)*(10 ** 12)


In [151]:
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)
# transform X86 into normal number
swaps["sqrtPriceX96"] = (swaps["sqrtPriceX96"] / (2**96) ) ** 2
# transform sqrt_p into p
swaps["sqrtPriceX96"] = 1 / (swaps["sqrtPriceX96"] * (10 ** 12))

# 首先不考虑tick被击穿的情况, 表中的tick是本次swap之后池子内的swap，为了先简化计算，假设最终的tick在我们选定的头寸之内的所有swaps都是在我们的头寸之内发生的
# 筛选swaps的条件：swap后最终的tick在我们的头寸tick之内
condition = (swaps['tick'] > tick_i_l) & (swaps['tick'] < tick_i_u)
# 筛选
print("total swaps: \n", swaps.shape[0])
swaps = swaps[condition]
print("total swaps in range: \n", swaps.shape[0])

this is the cursor: 0x3a901948bd17833d5ca03f77a39a77605f522826fc34429840f934dfc382e7f8#21706
this is the cursor: 0x74078b8e3b74856ac8349345c24bcece02a0b0a16ae17bcae6b24194a16e9192#25545
this is the cursor: 0xb1b958f22d4caae8b6bbb0d2e3e9e0a1d3212afb2fe2302f808623b179262f36#21701
this is the cursor: 0xec507425ff1cc482de9113b05a67760075f2ac296576f9176dd687077d64a621#23524
this is the cursor: 0xffd1e269f6bc10dab9aa66698e32dd5fc6bc4c09fee02af1499635fedfe908c0#26062
total swaps: 
 4317
total swaps in range: 
 1445


In [152]:
# 现在来加总然后计算手续费
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick
4,1681283663,252.343510,-254.978898,0.98917,-276216
7,1678997147,518.144934,-522.987248,0.990248,-276227
9,1679654111,1142.025901,-1155.629973,0.987736,-276201
10,1679653775,-2777.243446,2813.142400,0.987729,-276201
11,1680737939,-815.581234,825.000000,0.989077,-276215
15,1680221519,-13.630000,13.798111,0.988311,-276207


In [1]:
price_l

NameError: name 'price_l' is not defined

In [153]:
# 先把tick price转换成sqrtX96的形式
sqrt_p_lower = (price_l ** 0.5) * (2 ** 96)
sqrt_p_upper = (price_u ** 0.5) * (2 ** 96)
sqrt_p_lower_f = '{:.0f}'.format(sqrt_p_lower)
sqrt_p_upper_f = '{:.0f}'.format(sqrt_p_upper)

# 筛选出所有在tick之内的swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id, sqrtPriceX96_gte=sqrt_p_lower_f, sqrtPriceX96_lte=sqrt_p_upper_f)
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id, sqrtPrice_gt=sqrt_p_lower_f, sqrtPrice_lt=sqrt_p_upper_f)

this is the cursor: 0xb14697d320814ebfc49cacd10b377f8a284b3bf8d971279182ee904ea0683aec#23559
this is the cursor: 0xffd1e269f6bc10dab9aa66698e32dd5fc6bc4c09fee02af1499635fedfe908c0#26062
this is the cursor: 1681416000


In [154]:
liquidity_data = liquidity_data[["periodStartUnix", "liquidity"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "liquidity": "liquidity"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "liquidity"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,liquidity
0,0x102ab09e3db41935dafce3bdf85411d044c62607bfcd...,1678468679,0xe592427a0aece92de3edee1f18e0157c05861564,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0x9fea6d71bed43ab13cff10d8eeb9354d47f4e42c,-144802.531667347300083036,146197.450741,146197.450741,79639906560328829756323,-276221,13,113647072223080402568
1,0x3a901948bd17833d5ca03f77a39a77605f522826fc34...,1678470083,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,0xddcf904464166ac74629241ba21199adee3a04ad,2992.5,-3022.092718,3022.092718,79637799732160080764280,-276221,197,113647072223080402568
2,0x22f69e5a8ff3b13be9f66598730d2618697188606596...,1678471139,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,0x2ee36e41387f87b7e6f678a86d1e575b23b996f5,1881.061178865875054321,-1899.581078,1899.581078,79636475454152159045662,-276222,22,113647072223080402568


In [155]:
# 获取头寸的liquidity，后续手续费的分成是按照头寸的liquidity占总liquidity的比例来计算的
position_liquidity = position_data["liquidity"][0]
merged_data['liquidity'] = merged_data['liquidity'].astype(float)
merged_data['ratio'] = float(position_liquidity) / merged_data["liquidity"]
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)


254.59212998490977
231.42731550325948
